In [18]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, r2_score, accuracy_score
from sklearn.svm import  SVC
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, ElasticNet
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import warnings
warnings.simplefilter('ignore')
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, BaggingRegressor, RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor, GradientBoostingClassifier, StackingClassifier, RandomForestRegressor, StackingRegressor
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor, LGBMClassifier

In [8]:
train = pd.read_csv("./Loan Prediction/train_ctrUa4K.csv")
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [9]:
test = pd.read_csv("./Loan Prediction/test_lAUu6dG.csv")
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [10]:
train = train.drop('Loan_ID', axis=1)
test = test.drop('Loan_ID', axis=1)

In [17]:
train.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [11]:
train.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [19]:
train['Gender'].unique()

array(['Male', 'Female', nan], dtype=object)

In [23]:
X_train, y_train = train.drop('Loan_Status', axis=1), train['Loan_Status']

In [32]:
imp_const = SimpleImputer(strategy='constant', fill_value='Unknown')
imp_med = SimpleImputer(strategy='median')
ct_imp = make_column_transformer((imp_const, make_column_selector(dtype_include=object)),(imp_med, make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False).set_output(transform='pandas')
X_trn_imp = ct_imp.fit_transform(X_train)
X_tst_imp = ct_imp.transform(test)

In [36]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
ct_ohe = make_column_transformer((ohe, make_column_selector(dtype_include=object)),("passthrough", make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False).set_output(transform='pandas')

In [37]:
X_trn_ohe = ct_ohe.fit_transform(X_trn_imp)
X_tst_ohe = ct_ohe.transform(X_tst_imp)

In [38]:
X_trn_ohe.shape, X_tst_ohe.shape

((614, 18), (367, 18))

In [ ]:
rf = RandomForestClassifier(random_state=42, n_estimators=30)


In [49]:
pipe_rf = Pipeline([('IMP',ct_imp),('OHE',ct_ohe),('RF',rf)])
pipe_rf.fit(X_train, y_train)

Pipeline(steps=[('IMP',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(fill_value='Unknown',
                                                                strategy='constant'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000254B6A4B820>),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000254B6A4BD9...
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000254B6594940>),
                                                 ('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000254B6595660>)],
                                   verbose_feature_names_out=False)),
                ('RF', DecisionTreeClassifier(max_depth=8, random_state=42))])

In [50]:
y_pred = pipe_rf.predict(test)

In [51]:
submit = pd.read_csv("./Loan Prediction/sample_submission_49d68Cx.csv")
submit.head()

,Loan_ID,Loan_Status
0,LP001015,N
1,LP001022,N
2,LP001031,N
3,LP001035,N
4,LP001051,N


In [52]:
submit['Loan_Status'] = y_pred

In [53]:

submit.to_csv("./submitLoan.csv", index=False)